In [1]:
#import math
#import numpy as np
#import importlib
#import frodo
#from frodo import *

In [5]:
#importlib.reload(frodo)
#from frodo import *

In [3]:
import numpy as np
from math import log

class Frodo:
    def __init__(ps, n):
        if n == 640:
            ps.n = 640
            ps.m_bar = 8
            ps.n_bar = 8
            ps.agree_bits = 128
            ps.sigma = 2.8
            ps.logq = 15
            ps.B = 2
            ps.lenX = 16
            ps.chi_table = [9288, 8720, 7216, 5264, 3384, 1918, 958, 422, 164, 56, 17, 4, 1]
        elif n == 976:
            ps.n = 976
            ps.m_bar = 8
            ps.n_bar = 8
            ps.agree_bits = 192
            ps.sigma = 2.3
            ps.logq = 16
            ps.B = 3
            ps.lenX = 16
        elif n == 1344:
            ps.n = 1344
            ps.m_bar = 8
            ps.n_bar = 8
            ps.agree_bits = 256
            ps.sigma = 1.4
            ps.logq = 16
            ps.B = 4
            ps.lenX = 16
            ps.chi_table = [18286, 14320, 6876, 2023, 364, 40, 2]
        else:
            raise ValueError('unsupported parameter')

        ps.q = (1 << ps.logq)
        ps.required_coordinates = (ps.agree_bits + ps.B - 1) // ps.B
        ps.chi = {}
        for x, pr in enumerate(ps.chi_table):
            ps.chi[x] = ps.chi[-x] = 1.0*pr / (1 << 16)
            
        ps.upper_t = ps.q / (2 ** ps.B) / 2
        ps.lower_t = -ps.upper_t

NEGLIGIBLE = 1E-200
def valid_distr(d):
    """Check whether the dictionary represents a distribution.

    Args:
      d: A distribution represented as a dictionary.

    Returns:
      True iff d is a valid distribution.
    """

    return abs(sum(d.values()) - 1.) < 1E-9 and all(x >= 0 for x in d.values())
def filter_negl(A):
    return {v: p for (v, p) in A.items() if p >= NEGLIGIBLE}
def convolution(a, b, q=None):
    """Compute a convolution of two probability distributions.

    Args:
      a, b: Two distributions represented as dictionaries.
      q: If q is defined, computation is done modulo q. If q is None,
         distributions are taken over integers.

    Returns:
      A distribution represented as a dictionary.

    Raises:
      ValueError: if the input distributions are not valid.
    """

    def mod_to_int(x):
        return x if x < q / 2 else x - q

    def dict_to_arr(x):
        """Convert a distribution into an (offset, array) pair.
        """
        min_x, max_x = min(x.keys()), max(x.keys())
        a = np.zeros(max_x - min_x + 1, dtype=np.float128)
        for i, p in x.items():
            a[i - min_x] = p
        return min_x, a

    if not (valid_distr(a) and valid_distr(b)):
        print(sum(a.values()), sum(b.values()))
        raise ValueError('One of the input distributions is not valid')

    if q is not None:
        a = {mod_to_int(v): p for v, p in a.items()}
        b = {mod_to_int(v): p for v, p in b.items()}

    min_a, ar_a = dict_to_arr(a)
    min_b, ar_b = dict_to_arr(b)

    ar_c = np.convolve(ar_a, ar_b)
    c = {}
    for i, p in enumerate(ar_c):
        if p >= NEGLIGIBLE:
            v = i + min_a + min_b
            if q is not None:
                v = (v + q) % q
                c[v] = c.get(v, 0) + p  # add, in case there's wraparound
            else:
                c[v] = p
    return c
# def convolution(A, B, q = None):
#     C = {}
#     for a in A:
#         for b in B:
#             c = a+b
#             if q is not None:
#                 c = c % q
#             C[c] = C.get(c, 0) + A[a] * B[b]
#     return filter_negl(C)
def iter_convolution(A, k, q = None):
    B = {0: 1.0}
    k_bin = bin(k)[2:]
    for ch in k_bin:
        B = convolution(B, B, q)
        if ch == '1':
            B = convolution(B, A, q)
    return B
def pdf_product(A, B, q = None):
    C = {}
    for a, x in A.items():
        for b, y in B.items():
            c = a*b
            if q is not None:
                c %= q
            C[c] = C.get(c, 0) + x*y
    return filter_negl(C)
def pdf_hammer(A, h, force=1, logq=16):
    B = {}
    for x in A:
        xb = (x % (1 << logq)).to_bytes(logq/8, 'big')
        yb = bytearray(xb)
        hb = int(abs(h)).to_bytes(logq/8, 'big')
        if force==1:
            yb[0], yb[1] = yb[0] | hb[0], yb[1] | hb[1]
        else:
            yb[0], yb[1] = yb[0] & ~hb[0], yb[1] & ~hb[1]
        y = int.from_bytes(bytes(yb), 'big')
        if y >= (1 << (logq-1)):
            y = y - (1 << logq)
        B[y] = B.get(y, 0) + A[x]
    return B
def filter_distr(A, f, conditioning_pr=False):
    S = sum([p for x, p in A.items() if f(x)])
    B = {x: p/S for x, p in A.items() if f(x)}
    if conditioning_pr:
        return B, S
    return B
def map_distr(A, f):
    B = {}
    for x, p in A.items():
        y = f(x)
        B[y] = B.get(y, 0) + p
    return B

def hammer_dfr(Xp, Yp, k, X, Y, n, Z, lower_t, upper_t, q):
    '''
    Xp = [X1, X2, ...]
    Yp = [Y1, Y2, ...]
    k = [k1, k2, ...]
    
    a <- (X1*Y1)k1 + (X2*Y2)k2 + ... + (XY)n + Z
    
    return probability [lower_t <= v < upper_t]
    '''
    A = {0: 1.0}
    
    H = {0: 1.0}
    for i in range(len(Xp)):
        XiYiki = iter_convolution(pdf_product(Xp[i], Yp[i], q), k[i], q)
        H = convolution(H, XiYiki, q)
        
    H = convolution(H, Z, q)
    
    A = convolution(H, iter_convolution(pdf_product(X, Y, q), n, q), q)
        
    A = convolution(A, Z, q)
    
    lower_fail_pr = 0.0
    upper_fail_pr = 0.0
    for x, p in A.items():
        y = x if x < q / 2 else x - q 
        if y >= upper_t:
            upper_fail_pr += p
        if y < lower_t:
            lower_fail_pr += p
    fail_pr = lower_fail_pr + upper_fail_pr
    
#     avg = 0.0
#     m = 9999999
#     M = -9999999
#     for x, p in H.items():
#         y = x if x < q / 2 else x - q
#         avg += y*p
#         m = min(m, y)
#         M = max(M, y)
    
#     upper_t_max = upper_t - m
#     upper_t_min = upper_t - M
#     upper_t_avg = upper_t - avg
    
#     lower_t_max = lower_t - m
#     lower_t_min = lower_t - M
#     lower_t_avg = lower_t - avg
    
    o = lambda: 1
    o.lower_fail_pr = lower_fail_pr
    o.upper_fail_pr = upper_fail_pr
#     o.lower_t_min = lower_t_min
#     o.lower_t_max = lower_t_max
#     o.lower_t_avg = lower_t_avg
#     o.upper_t_min = upper_t_min
#     o.upper_t_max = upper_t_max
#     o.upper_t_avg = upper_t_avg
    o.total_fail_pr = lower_fail_pr + upper_fail_pr
    
    return o

def distr_min(d, q=None):
    m = 99999999
    for x, p in d.items():
        y  = x
        if q is not None:
            y = x if x < q/2 else x - q
        if p > 0:
            m = min(y, m)
    return m
def distr_max(d, q=None):
    M = -99999999
    for x, p in d.items():
        y  = x
        if q is not None:
            y = x if x < q/2 else x - q
        if p > 0:
            M = max(y, M)
    return M
def distr_avg(d, q=None):
    avg = 0.0
    for x, p in d.items():
        y  = x
        if q is not None:
            y = x if x < q/2 else x - q
        avg += y*p
    return avg
def Frodo640_hammer_dfr():
    ps = Frodo(640)
    for x in [256,128,64]:
        for y in [12,11,10]:
            Xp = map_distr(ps.chi, lambda v: v + x if v >= 0 else v)
            Xpp, C = filter_distr(Xp, lambda v: v >= 0, True)
            #C = conditioning pr for attack
            f = lambda v: abs(v) >= y
            Yp, S = filter_distr(ps.chi, f, True)
            #S = probability of passing filter
            Ypp = filter_distr(Yp, lambda v: v >= 0)
            Ypn = filter_distr(Yp, lambda v: v < 0)
            o = hammer_dfr([Xpp], [Yp], [1], ps.chi, ps.chi, 2*ps.n - 1, ps.chi, ps.lower_t, ps.upper_t, ps.q)
            p = hammer_dfr([Xp], [Yp], [1], ps.chi, ps.chi, 2*ps.n - 1, ps.chi, ps.lower_t, ps.upper_t, ps.q)
            o_honest = hammer_dfr([Xpp], [ps.chi], [1], ps.chi, ps.chi, 2*ps.n - 1, ps.chi, ps.lower_t, ps.upper_t, ps.q)
            p_honest = hammer_dfr([Xp], [ps.chi], [1], ps.chi, ps.chi, 2*ps.n - 1, ps.chi, ps.lower_t, ps.upper_t, ps.q)
            
            Tp = convolution(pdf_product(Xpp, Ypp, ps.q), ps.chi, ps.q)
            mp = distr_min(Tp, ps.q)
            Mp = distr_max(Tp, ps.q)
            avgp = distr_avg(Tp, ps.q)
            
            Tn = convolution(pdf_product(Xpp, Ypn, ps.q), ps.chi, ps.q)
            mn = distr_min(Tn, ps.q)
            Mn = distr_max(Tn, ps.q)
            avgn = distr_avg(Tn, ps.q)
            
            o.upper_t_max = ps.upper_t - mp
            o.upper_t_min = ps.upper_t - Mp
            o.upper_t_avg = ps.upper_t - avgp
    
            o.lower_t_max = ps.lower_t - mn
            o.lower_t_min = ps.lower_t - Mn
            o.lower_t_avg = ps.lower_t - avgn
            
            print("============================================================")
            print(f"Hammer value = {x}")
            print(f"Filtering value = {y}")
            print(f"Conditioning pr = 2^{log(C,2)}")
            print(f"Filtering pr = 2^{log(S,2)}")
            print("Conditional on hammered coordinate being positive: ---------")
            print(f"Upper fail pr = 2^{log(o.upper_fail_pr, 2)}")
            print(f"Lower fail pr = 2^{log(o.lower_fail_pr, 2)}")
            print(f"Adversarial total fail pr = 2^{log(o.total_fail_pr, 2)}")
            print(f"Honest user fail pr = 2^{log(o_honest.total_fail_pr, 2)}")
            print(f"Upper t= {o.upper_t_min} , {o.upper_t_avg} , {o.upper_t_max}")
            print(f"Lower t= {o.lower_t_min} , {o.lower_t_avg} , {o.lower_t_max}")
            print("No Conditional: --------------------------------------------")
            print(f"Upper fail pr = 2^{log(p.upper_fail_pr, 2)}")
            print(f"Lower fail pr = 2^{log(p.lower_fail_pr, 2)}")
            print(f"Adversarial total fail pr = 2^{log(p.total_fail_pr, 2)}")
            print(f"Honest user fail pr = 2^{log(p_honest.total_fail_pr, 2)}")
            #print(f"Upper t= {p.upper_t_min} , {p.upper_t_avg} , {p.upper_t_max}")
            #print(f"Lower t= {p.lower_t_min} , {p.lower_t_avg} , {p.lower_t_max}")
            print("============================================================")

In [4]:
Frodo640_hammer_dfr()

Hammer value = 256
Filtering value = 12
Conditioning pr = 2^-0.8087865277787137
Filtering pr = 2^-15.0
Conditional on hammered coordinate being positive: ---------
Upper fail pr = 2^-13.139630319083203
Lower fail pr = 2^-13.158615321107511
Adversarial total fail pr = 2^-12.149091591414745
Honest user fail pr = 2^-27.017821553100088
Upper t= 868.0 , 1000.6447129263337 , 1036.0
Lower t= -1036.0 , -1000.6447129263337 , -868.0
No Conditional: --------------------------------------------
Upper fail pr = 2^-13.948416846861917
Lower fail pr = 2^-13.967401848886228
Adversarial total fail pr = 2^-12.957878119193458
Honest user fail pr = 2^-27.8266080808788
Hammer value = 256
Filtering value = 11
Conditioning pr = 2^-0.8087865277787137
Filtering pr = 2^-12.678071905112638
Conditional on hammered coordinate being positive: ---------
Upper fail pr = 2^-15.33562915615374
Lower fail pr = 2^-15.354968768730375
Adversarial total fail pr = 2^-14.345266556267887
Honest user fail pr = 2^-27.0178215531000

In [4]:
print(f"asdfsdf {log(10.0,2)}")

asdfsdf 3.32192809488736


In [17]:
X = ps.chi
Xp = map_distr(ps.chi, lambda x: x + 256 if x >=0 else x)
Yp = filter_distr(ps.chi, lambda x: abs(x) >= 12)

In [19]:
T = convolution(X, pdf_product(Xp, Yp, ps.q), ps.q)

In [20]:
T

{29540: 1.1641532182693481445e-10,
 29541: 4.656612873077392578e-10,
 29542: 1.9790604710578918457e-09,
 29543: 6.5192580223083496094e-09,
 29544: 1.909211277961730957e-08,
 29545: 4.91272658109664917e-08,
 29546: 1.11525878310203552246e-07,
 29547: 2.2328458726406097412e-07,
 29548: 3.939494490623474121e-07,
 29549: 6.128102540969848633e-07,
 29550: 8.400529623031616211e-07,
 29551: 1.015141606330871582e-06,
 29552: 1.0817311704158782959e-06,
 29553: 1.0170042514801025391e-06,
 29554: 8.479692041873931885e-07,
 29555: 6.388872861862182617e-07,
 29556: 4.703179001808166504e-07,
 29557: 4.1979365050792694092e-07,
 29558: 5.5762939155101776123e-07,
 29559: 9.422656148672103882e-07,
 29560: 1.594889909029006958e-06,
 29561: 2.4577602744102478027e-06,
 29562: 3.3621909096837043762e-06,
 29563: 4.0610320866107940674e-06,
 29564: 4.3271575123071670532e-06,
 29565: 4.0684826672077178955e-06,
 29566: 3.3938558772206306458e-06,
 29567: 2.5620684027671813965e-06,
 29568: 1.9003637135028839111e-0

In [21]:
m = 99999999
M = -99999999
avg = 0
for x, p in T.items():
    m = min(m, x)
    M = max(M, x)
    avg += x*p

In [22]:
m, M, avg

(0, 32767, 16383.96673583984375)

In [4]:
hp = {(256, lambda x: abs(x)==11): 1}
log(Frodo_hammer_dfr(ps, hp),  2)

-18.416062176339054

In [13]:
f = lambda x: abs(x) == 12
hp = {(256, f): 1}
a = log(sum([pr for x, pr in ps.chi.items() if f(x)]), 2)
b = log(Frodo_hammer_dfr(ps, hp), 2) 
print(a, b, a + b)

-15.0 -12.949340862554823 -27.949340862554823


In [8]:
hpa = {(256, lambda x: abs(x) >= 12): 1}
log(Frodo_hammer_dfr(ps, hpa), 2)

-12.958802980741906

In [7]:
hph = {(256, None): 1}
log(Frodo_hammer_dfr(ps, hph), 2)

-27.827578047191004

In [20]:
f = lambda x: abs(x) >= v
print(sum([pr for x, pr in Frodo640.chi.items() if f(x)]) )
log(Frodo640.chi[12] + Frodo640.chi[-12], 2)

3.0517578125e-05


-15.0

In [11]:
for v, pr in sorted(list(Frodo640.chi.items())):
    if v < 0:
        continue
    print(v, pr * 2**16 )

0 9288.0
1 8720.0
2 7216.0
3 5264.0
4 3384.0
5 1918.0
6 958.0
7 422.0
8 164.0
9 56.0
10 17.0
11 4.0
12 1.0


In [36]:
l = [9288, 8720, 7216, 5264, 3384, 1918, 958, 422, 164, 56, 17, 4, 1]
for x, pr in enumerate(l):
    print(pr* 2**(-16), Frodo640.chi[x])
    print( pr* 2**(-16) == Frodo640.chi[x] )

0.1417236328125 0.1417236328125
True
0.133056640625 0.133056640625
True
0.110107421875 0.110107421875
True
0.080322265625 0.080322265625
True
0.0516357421875 0.0516357421875
True
0.029266357421875 0.029266357421875
True
0.014617919921875 0.014617919921875
True
0.006439208984375 0.006439208984375
True
0.00250244140625 0.00250244140625
True
0.0008544921875 0.0008544921875
True
0.0002593994140625 0.0002593994140625
True
6.103515625e-05 6.103515625e-05
True
1.52587890625e-05 1.52587890625e-05
True


In [5]:
(-32508) % ps.q

260

In [6]:
33028 % ps.q

260

In [12]:
12*256, 6*128*2

(3072, 1536)

In [9]:
def optimize_filter(ps, hammer_pattern, one_side = True):
    from collections.abc import Iterable  
    dfr_overall = 0
    opt_t = 1
    hammer_filter_pattern = {}
    for t in [x for x in ps.chi if x > 0]:
        hammer_filter_pattern_t = {}
        if one_side:
            pr_filter_single = sum([v for (x, v) in ps.chi.items() if x >= t])
        else:
            pr_filter_single = sum([v for (x, v) in ps.chi.items() if abs(x) >= t])
        f1 = lambda x: x >= t
        f2 = lambda x: x <= -t
        for (p, k) in hammer_pattern.items():
            s = p
            if isinstance(p, Iterable):
                s = sum(p)
            if s >= 0:
                f = f1
            else:
                f = f2
            if not one_side:
                f = lambda x: abs(x) >= t
            hammer_filter_pattern_t[(p, f)] = k
        pr_filter = pr_filter_single ** sum(hammer_pattern.values())
        dfr = Frodo_hammer_dfr(ps, hammer_filter_pattern)
        dfr_overall_t = pr_filter*dfr
        print('t =', t)
        print('dfr =', log(dfr, 2))
        print('pr_filter =', log(pr_filter, 2) )
        print('dfr_overall_t =', log(dfr_overall_t,2) )
        if dfr_overall_t  > dfr_overall:
            dfr_overall = dfr_overall_t
            opt_t = t
            hammer_filter_pattern = hammer_filter_pattern_t
    return (opt_t, hammer_filter_pattern)

In [49]:
hp = {32:10}

In [10]:
t, hfp = optimize_filter(Frodo640, {32:40})
t

t = 1
dfr = -144.72815857337122
pr_filter = -48.81943283425998
dfr_overall_t = -193.54759140763122
t = 2
dfr = -10.111535101403133
pr_filter = -70.23734143220729
dfr_overall_t = -80.34887653361042
t = 3
dfr = -5.140638999691546
pr_filter = -97.07304861254057
dfr_overall_t = -102.21368761223214
t = 4
dfr = -2.2296735465564725
pr_filter = -129.70439962533445
dfr_overall_t = -131.93407317189093
t = 5
dfr = -0.8483382481482352
pr_filter = -168.4186542011856
dfr_overall_t = -169.26699244933386
t = 6
dfr = -0.2927923867924131
pr_filter = -213.45767583130907
dfr_overall_t = -213.7504682181015
t = 7
dfr = -0.09583190063210947
pr_filter = -264.998422746123
dfr_overall_t = -265.09425464675513
t = 8
dfr = -0.03124341079627863
pr_filter = -323.24547050901623
dfr_overall_t = -323.2767139198125
t = 9
dfr = -0.010574322237264497
pr_filter = -388.58391124551
dfr_overall_t = -388.59448556774737
t = 10
dfr = -0.003847502017878792
pr_filter = -461.62273525450814
dfr_overall_t = -461.626582756526
t = 11
d

2

In [56]:
t, hfp = optimize_filter(Frodo640, {32:20}, False)
t

t = 1
dfr = -144.72815857337122
pr_filter = -4.409716417129987
dfr_overall_t = -149.13787499050122
t = 2
dfr = -50.27822105142728
pr_filter = -15.118670716103644
dfr_overall_t = -65.39689176753093
t = 3
dfr = -42.58587587252516
pr_filter = -28.536524306270287
dfr_overall_t = -71.12240017879544
t = 4
dfr = -34.87730315730652
pr_filter = -44.85219981266723
dfr_overall_t = -79.72950296997377
t = 5
dfr = -28.00454279720751
pr_filter = -64.2093271005928
dfr_overall_t = -92.21386989780031
t = 6
dfr = -22.43544443026673
pr_filter = -86.72883791565454
dfr_overall_t = -109.16428234592127
t = 7
dfr = -18.168881114963373
pr_filter = -112.4992113730615
dfr_overall_t = -130.66809248802488
t = 8
dfr = -14.961224682190236
pr_filter = -141.62273525450811
dfr_overall_t = -156.58395993669836
t = 9
dfr = -12.544063806167502
pr_filter = -174.29195562275504
dfr_overall_t = -186.83601942892253
t = 10
dfr = -10.71455383759565
pr_filter = -210.81136762725407
dfr_overall_t = -221.52592146484972
t = 11
dfr = -9

2

In [37]:
list(hfp.keys())[0][1](11)

False

In [1]:
#Frodo640
sigma = 2.8
nbar = mbar = 8
B = w = 2
n = 640
q = (1 << 15)
qlog = 15
ubits = len_chi = 16
agree_bits = 128
required_coordinates = (agree_bits + w - 1)//w
samples = 2 * (mbar + nbar) * n + required_coordinates
max_m = n + max(mbar, nbar)

In [57]:
Frodo640.chi

{1: 0.133056640625,
 -1: 0.133056640625,
 2: 0.110107421875,
 -2: 0.110107421875,
 3: 0.080322265625,
 -3: 0.080322265625,
 0: 0.1417236328125,
 4: 0.0516357421875,
 -4: 0.0516357421875,
 5: 0.029266357421875,
 -5: 0.029266357421875,
 6: 0.014617919921875,
 -6: 0.014617919921875,
 7: 0.006439208984375,
 -7: 0.006439208984375,
 8: 0.00250244140625,
 -8: 0.00250244140625,
 9: 0.0008544921875,
 -9: 0.0008544921875,
 10: 0.0002593994140625,
 -10: 0.0002593994140625,
 11: 6.103515625e-05,
 -11: 6.103515625e-05,
 12: 1.52587890625e-05,
 -12: 1.52587890625e-05}

In [7]:
attacks = [svp_classical, svp_plausible]

In [10]:
costs_gauss_single_shot = estimate_cost(2 ** qlog, n, max_m, sigma, attacks)

In [11]:
costs_gauss_single_shot

[149.59337450720804, 108.72641166033193, inf]

In [12]:
c = costs_gauss_single_shot[0]

In [13]:
Frodo976 = 

145.59337450720804

In [14]:
D = approximate_dgauss(sigma, samples, base_security, None, ubits)

In [5]:
8 * 8 * 2

128

In [30]:
3+             7+15+ 8+4+7 +5

49

In [34]:
d = {1: lambda x: x + 1}

In [36]:
d[1](3)

4

In [37]:
d = {(1,2): 3}

In [38]:
g = {x: 4 for x,_ in d}

In [39]:
g

{1: 4}

In [45]:
for x,y, z in d.items():
    print(x, y, z)

ValueError: not enough values to unpack (expected 3, got 2)

In [42]:
d.items()

dict_items([((1, 2), 3)])

In [6]:
20 + 2 + 1 + 3 + 7 +7 + 4 + 1

45

In [7]:
'wigglyboi'[::-1]

'iobylggiw'